# 

In [2]:
!pip install -U openai datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 13.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 

In [3]:
import os
import openai

openai.api_key = "YOUR_OPENAI_KEY" 


In [4]:
from huggingface_hub import login

# token Huggingface
login(token="YOUR_HF_KEY") 

In [5]:
from datasets import load_dataset

# Load tập huấn luyện từ HuggingFace
ds = load_dataset("5CD-AI/Viet-Resume-VQA", split="train")
print(f"Số dòng: {len(ds)}")
ds[0]


README.md:   0%|          | 0.00/970 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/126M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2389 [00:00<?, ? examples/s]

Số dòng: 2389


{'id': 0,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=729x1118>,
 'description': 'Bức ảnh là một CV (Curriculum Vitae) với bố cục đơn giản. Bên trái là phần thông tin cá nhân và kinh nghiệm làm việc, bên phải là phần thông tin liên lạc, mục tiêu nghề nghiệp, kỹ năng và chứng chỉ. Các thông tin được trình bày trong các khung màu trắng nền hồng, mỗi khung được phân cách bởi một đường kẻ màu đen mỏng. Các dòng chữ được căn chỉnh theo lề trái, font chữ đen, kích thước chữ rõ ràng.',
 'extractions': "{'Họ và tên': 'NGUYỄN MINH TRIỆT', 'Chức danh': 'Bao trì', 'Ngày sinh': '20/2/1994', 'Giới tính': 'Nam', 'Số điện thoại': '0372052463', 'Email': 'Nguyentrjtet94tv@gmail.com', 'Địa chỉ': 'Ngãi trung-tập ngãi-tiểu cần-trà vinh', 'Trình độ học vấn': 'Cao đẳng nghề trà vinh', 'Xếp loại': 'khá', 'Kinh nghiệm làm việc': ['Cửa hàng điện cơ tân hiên', 'Sửa chữa các loại mô tô 1 pha 3 pha.dụng cụ cầm tay', 'Thiết kế và lắp ráp tủ điện(tự điều khiển)', 'Công ty điện lực thành phố trà

In [6]:
import random

# Sampling bao nhiêu dòng/2389 dòng thì làm ở đây, recommend 2389 cho nhiều, lấy 10 max workers ở dưới
sampled_data = random.sample(list(ds), k=2389)

# Lấy cột 'extractions'
cv_texts = [row["extractions"] for row in sampled_data if row["extractions"].strip() != ""]
print(f"Số CV hợp lệ: {len(cv_texts)}")


Số CV hợp lệ: 2389


In [7]:
import random

# Danh sách instruction cố định
INSTRUCTIONS = [
    "CV này có được trình bày chuyên nghiệp và rõ ràng không?",
    "CV này còn thiếu thông tin quan trọng nào mà ứng viên nên bổ sung?",
    "Dựa trên thông tin CV, hãy gợi ý 5 vị trí công việc phù hợp nhất với ứng viên này.",
    "Điểm mạnh nổi bật nhất của ứng viên là gì?",
    "Ứng viên có thể cải thiện những điểm nào để CV chuyên nghiệp hơn?",
    "CV có lỗi chính tả hoặc định dạng nào đáng chú ý không?",
    "CV này có đủ sức thuyết phục để nhà tuyển dụng mời ứng viên phỏng vấn không? Vì sao?",
    "Dựa vào nội dung CV, bạn đánh giá năng lực chuyên môn thực tế của ứng viên như thế nào?",
    "Ứng viên này phù hợp hơn với môi trường làm việc startup hay tập đoàn? Vì sao?",
    "Ứng viên có đạt được thành tựu cụ thể và có thể đo lường được trong công việc không?",
    "CV này thể hiện ứng viên chỉ hoàn thành nhiệm vụ hay có chủ động tạo ra giá trị và đổi mới?",
    "CV có cho thấy sự phát triển về trách nhiệm hoặc vai trò theo thời gian không?",
    "Ứng viên có kết hợp tốt kỹ năng cứng và kỹ năng mềm để thích nghi với môi trường làm việc không?",
    "CV có thể hiện ứng viên đến từ cùng ngành hay chuyển ngành? Điều này ảnh hưởng thế nào đến khả năng thích nghi?"
]

def prompt_cv_eval_multi(cv_text, instruction_list):
    instructions = random.sample(instruction_list, k=random.randint(3, 5))
    joined_instructions = "\n".join([f"- {ins}" for ins in instructions])
    
    prompt = f"""
Bạn là chuyên gia tuyển dụng. Dưới đây là một CV được trích xuất từ ảnh.

Hãy trả lời các câu hỏi đánh giá dưới đây, mỗi câu hỏi trả lời bằng một JSON object gồm:
- "instruction": câu hỏi
- "input": nội dung CV (sao y như dưới)
- "output": câu trả lời dựa trên nội dung CV (chuyên nghiệp, đúng trọng tâm)

Yêu cầu:
- Không bịa thông tin không có trong CV
- Phản hồi chính xác, đúng vai trò người tuyển dụng
- Trả về đúng mảng JSON [{{}}]

CV:
\"\"\"{cv_text}\"\"\"

Câu hỏi đánh giá:
{joined_instructions}
"""
    return prompt


In [8]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
import json, re, time

client = OpenAI(api_key=openai.api_key)

def safe_json_list_parse(text):
    try:
        data = json.loads(text)
        if isinstance(data, list):
            return data
    except:
        match = re.search(r'\[.*\]', text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except:
                return None
    return None

def gpt_multi_eval(cv_text):
    try:
        prompt = prompt_cv_eval_multi(cv_text, INSTRUCTIONS)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=1024
        )
        data = safe_json_list_parse(response.choices[0].message.content.strip())
        if data:
            for item in data:
                item["input"] = cv_text  # Gắn lại input chuẩn là CV gốc
        return data
    except Exception as e:
        print(f"❌ GPT Error: {e}")
        return None

results = []

with ThreadPoolExecutor(max_workers=10) as executor:  # số luồng 
    futures = [executor.submit(gpt_multi_eval, cv) for cv in cv_texts[:2389]]  
    for i, f in enumerate(as_completed(futures)):
        r = f.result()
        if r:
            if len(r) < 2:
                print(f"⚠️ CV {i+1}: Chỉ có {len(r)} câu hỏi < 2 → Bỏ qua")
                continue
            r = r[:13]  # Giới hạn tối đa 13 câu hỏi
            results.extend(r)
            print(f"✅ CV {i+1}: Thêm {len(r)} câu hỏi")
        else:
            print(f"⚠️ Bỏ qua CV {i+1}")

print(f"\n✅ Tổng số mẫu sinh ra: {len(results)}")


⚠️ Bỏ qua CV 1
⚠️ Bỏ qua CV 2
⚠️ Bỏ qua CV 3
⚠️ Bỏ qua CV 4
⚠️ Bỏ qua CV 5
✅ CV 6: Thêm 5 câu hỏi
⚠️ Bỏ qua CV 7
⚠️ Bỏ qua CV 8
⚠️ Bỏ qua CV 9
✅ CV 10: Thêm 3 câu hỏi
✅ CV 11: Thêm 3 câu hỏi
✅ CV 12: Thêm 5 câu hỏi
✅ CV 13: Thêm 5 câu hỏi
⚠️ Bỏ qua CV 14
✅ CV 15: Thêm 3 câu hỏi
⚠️ Bỏ qua CV 16
⚠️ Bỏ qua CV 17
⚠️ Bỏ qua CV 18
✅ CV 19: Thêm 3 câu hỏi
✅ CV 20: Thêm 4 câu hỏi
⚠️ Bỏ qua CV 21
✅ CV 22: Thêm 5 câu hỏi
⚠️ Bỏ qua CV 23
⚠️ Bỏ qua CV 24
⚠️ Bỏ qua CV 25
⚠️ Bỏ qua CV 26
✅ CV 27: Thêm 4 câu hỏi
✅ CV 28: Thêm 4 câu hỏi
✅ CV 29: Thêm 5 câu hỏi
✅ CV 30: Thêm 4 câu hỏi
✅ CV 31: Thêm 3 câu hỏi
✅ CV 32: Thêm 3 câu hỏi
⚠️ Bỏ qua CV 33
✅ CV 34: Thêm 5 câu hỏi
✅ CV 35: Thêm 3 câu hỏi
⚠️ Bỏ qua CV 36
⚠️ Bỏ qua CV 37
✅ CV 38: Thêm 4 câu hỏi
⚠️ Bỏ qua CV 39
✅ CV 40: Thêm 3 câu hỏi
✅ CV 41: Thêm 5 câu hỏi
⚠️ Bỏ qua CV 42
✅ CV 43: Thêm 5 câu hỏi
✅ CV 44: Thêm 3 câu hỏi
✅ CV 45: Thêm 4 câu hỏi
⚠️ Bỏ qua CV 46
⚠️ Bỏ qua CV 47
⚠️ Bỏ qua CV 48
✅ CV 49: Thêm 5 câu hỏi
✅ CV 50: Thêm 5 câu hỏi
⚠️ Bỏ qua

In [9]:
with open("train.jsonl", "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print("✅ Đã lưu train.jsonl thành công.")


✅ Đã lưu train.jsonl thành công.
